In [12]:
import gurobipy as gp
from gurobipy import GRB

In [14]:
# Sets and parameters (example data)
m = 3  # number of ingredients
n = 2  # number of products
b = [100, 150, 200]  # availability of ingredients
c = [2, 3, 1]  # cost per kilogram of ingredients
d = [50, 60]  # minimum production requirement for products
e = [10, 15]  # revenue per kilogram of products
l = [[0.2, 0.1], [0.3, 0.4], [0.1, 0.2]]  # lower bound on fractions
u = [[0.8, 0.7], [0.5, 0.6], [0.4, 0.5]]  # upper bound on fractions

In [16]:
# Create a new model
model = gp.Model("Mixitup")

# Decision variables
x = model.addVars(n, vtype=GRB.CONTINUOUS, name="x")  # amount of each product
y = model.addVars(m, n, vtype=GRB.CONTINUOUS, name="y")  # amount of each ingredient in each product

In [18]:
# Objective: Maximize profit
profit = gp.quicksum(e[j] * x[j] for j in range(n)) - gp.quicksum(c[i] * y[i, j] for i in range(m) for j in range(n))
model.setObjective(profit, GRB.MAXIMIZE)

In [20]:
# Constraints
# Ingredient availability
for i in range(m):
    model.addConstr(gp.quicksum(y[i, j] for j in range(n)) <= b[i], name=f"availability_{i}")

# Production requirement
for j in range(n):
    model.addConstr(x[j] >= d[j], name=f"production_{j}")

# Ingredient composition bounds
for i in range(m):
    for j in range(n):
        model.addConstr(y[i, j] >= l[i][j] * x[j], name=f"lower_bound_{i}_{j}")
        model.addConstr(y[i, j] <= u[i][j] * x[j], name=f"upper_bound_{i}_{j}")


In [22]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 24.0.0 24A335)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 8 columns and 32 nonzeros
Model fingerprint: 0xbb9389f9
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+02]
Presolve removed 15 rows and 6 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2490000e+04   5.060807e+02   0.000000e+00      0s
       3    4.9525000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.05 seconds (0.00 work units)
Optimal objective  4.952500000e+03


In [24]:
# Output the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    for j in range(n):
        print(f"Product {j}: {x[j].x} kg produced")
    for i in range(m):
        for j in range(n):
            print(f"Ingredient {i} in product {j}: {y[i, j].x} kg used")
else:
    print("No optimal solution found.")

Optimal solution found.
Product 0: 50.0 kg produced
Product 1: 337.5 kg produced
Ingredient 0 in product 0: 10.0 kg used
Ingredient 0 in product 1: 33.75 kg used
Ingredient 1 in product 0: 15.0 kg used
Ingredient 1 in product 1: 135.0 kg used
Ingredient 2 in product 0: 5.0 kg used
Ingredient 2 in product 1: 67.5 kg used
